In [3]:
import pandas as pd
import numpy as np

# модуль sparse библиотеки scipy понадобится для работы с разреженными матрицами 
from scipy.sparse import csr_matrix

from sklearn.neighbors import NearestNeighbors

anime = pd.read_csv('anime.csv')
ratings = pd.read_csv('rating.csv')

In [4]:
anime.head(20)

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
5,32935,Haikyuu!!: Karasuno Koukou VS Shiratorizawa Ga...,"Comedy, Drama, School, Shounen, Sports",TV,10,9.15,93351
6,11061,Hunter x Hunter (2011),"Action, Adventure, Shounen, Super Power",TV,148,9.13,425855
7,820,Ginga Eiyuu Densetsu,"Drama, Military, Sci-Fi, Space",OVA,110,9.11,80679
8,15335,Gintama Movie: Kanketsu-hen - Yorozuya yo Eien...,"Action, Comedy, Historical, Parody, Samurai, S...",Movie,1,9.10,72534
9,15417,Gintama&#039;: Enchousen,"Action, Comedy, Historical, Parody, Samurai, S...",TV,13,9.11,81109


In [5]:
ratings.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [6]:
anime.drop(['genre','type','episodes','rating','members'], axis = 1, inplace = True)

In [7]:
# Количество уникальных пользователей и аниме
print("Уникальных пользователей:", ratings['user_id'].nunique())
print("Уникальных аниме:", ratings['anime_id'].nunique())

Уникальных пользователей: 73515
Уникальных аниме: 11200


In [49]:
active_users = ratings['user_id'].value_counts()
active_users = active_users[active_users > 100].index  # например, пользователи с >100 оценками
rating_filtered = ratings[ratings['user_id'].isin(active_users)]
user_item_matrix = rating_filtered.pivot_table(index='anime_id', columns='user_id', values='rating', aggfunc='mean')

In [50]:
user_item_matrix.head()

user_id,1,5,7,11,13,14,17,21,29,35,...,73494,73495,73499,73500,73502,73503,73504,73507,73510,73515
anime_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,9.0,NaN,NaN,...,NaN,10.0,9.0,NaN,NaN,9.0,10.0,9.0,NaN,10.0
5,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,9.0,NaN,NaN,...,NaN,8.0,NaN,NaN,NaN,7.0,10.0,8.0,NaN,10.0
6,NaN,8.0,NaN,NaN,-1.0,NaN,7.0,NaN,NaN,NaN,...,9.0,-1.0,9.0,NaN,NaN,9.0,9.0,9.0,NaN,10.0
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,7.0,NaN,NaN,9.0,NaN,7.0,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
#замена пропущенных значений 
user_item_matrix.fillna(0, inplace = True)
print(user_item_matrix.shape)
user_item_matrix.head()

(11196, 24677)


user_id,1,5,7,11,13,14,17,21,29,35,...,73494,73495,73499,73500,73502,73503,73504,73507,73510,73515
anime_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,9.0,0.0,0.0,...,0.0,10.0,9.0,0.0,0.0,9.0,10.0,9.0,0.0,10.0
5,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,9.0,0.0,0.0,...,0.0,8.0,0.0,0.0,0.0,7.0,10.0,8.0,0.0,10.0
6,0.0,8.0,0.0,0.0,-1.0,0.0,7.0,0.0,0.0,0.0,...,9.0,-1.0,9.0,0.0,0.0,9.0,9.0,9.0,0.0,10.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,7.0,0.0,0.0,9.0,0.0,7.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
# сделаем то же самое
anime_votes = ratings.groupby('anime_id')['rating'].agg('count')
 
anime_mask = anime_votes[anime_votes > 50].index
 
# применим и отберем аниме с достаточным количеством оценок
user_item_matrix = user_item_matrix.loc[anime_mask,:]
 

print(user_item_matrix.shape)
user_item_matrix.head()

(5625, 24677)


user_id,1,5,7,11,13,14,17,21,29,35,...,73494,73495,73499,73500,73502,73503,73504,73507,73510,73515
anime_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,9.0,0.0,0.0,...,0.0,10.0,9.0,0.0,0.0,9.0,10.0,9.0,0.0,10.0
5,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,9.0,0.0,0.0,...,0.0,8.0,0.0,0.0,0.0,7.0,10.0,8.0,0.0,10.0
6,0.0,8.0,0.0,0.0,-1.0,0.0,7.0,0.0,0.0,0.0,...,9.0,-1.0,9.0,0.0,0.0,9.0,9.0,9.0,0.0,10.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,7.0,0.0,0.0,9.0,0.0,7.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
csr_data = csr_matrix(user_item_matrix.values)
print(csr_data)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 6034235 stored elements and shape (5625, 24677)>
  Coords	Values
  (0, 4)	-1.0
  (0, 7)	9.0
  (0, 12)	10.0
  (0, 14)	10.0
  (0, 15)	10.0
  (0, 16)	-1.0
  (0, 21)	10.0
  (0, 32)	9.0
  (0, 33)	10.0
  (0, 36)	10.0
  (0, 39)	9.0
  (0, 40)	8.0
  (0, 44)	7.0
  (0, 45)	8.0
  (0, 49)	10.0
  (0, 51)	8.0
  (0, 52)	10.0
  (0, 55)	8.0
  (0, 56)	9.0
  (0, 62)	10.0
  (0, 64)	9.0
  (0, 65)	-1.0
  (0, 67)	-1.0
  (0, 69)	7.0
  (0, 71)	-1.0
  :	:
  (5624, 24344)	9.0
  (5624, 24346)	-1.0
  (5624, 24360)	10.0
  (5624, 24384)	-1.0
  (5624, 24386)	10.0
  (5624, 24392)	8.0
  (5624, 24393)	-1.0
  (5624, 24404)	8.0
  (5624, 24424)	-1.0
  (5624, 24427)	8.0
  (5624, 24430)	8.0
  (5624, 24445)	-1.0
  (5624, 24451)	9.0
  (5624, 24460)	8.0
  (5624, 24469)	7.0
  (5624, 24470)	8.0
  (5624, 24472)	-1.0
  (5624, 24481)	9.0
  (5624, 24494)	-1.0
  (5624, 24542)	8.0
  (5624, 24547)	-1.0
  (5624, 24558)	8.0
  (5624, 24580)	10.0
  (5624, 24584)	8.0
  (5624, 24632

In [54]:
user_item_matrix = user_item_matrix.rename_axis(None, axis = 1).reset_index()
user_item_matrix.head()

,anime_id,1,5,7,11,13,14,17,21,29,...,73494,73495,73499,73500,73502,73503,73504,73507,73510,73515
0,1,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,9.0,0.0,...,0.0,10.0,9.0,0.0,0.0,9.0,10.0,9.0,0.0,10.0
1,5,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,9.0,0.0,...,0.0,8.0,0.0,0.0,0.0,7.0,10.0,8.0,0.0,10.0
2,6,0.0,8.0,0.0,0.0,-1.0,0.0,7.0,0.0,0.0,...,9.0,-1.0,9.0,0.0,0.0,9.0,9.0,9.0,0.0,10.0
3,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,7.0,0.0,0.0,9.0,0.0,7.0,0.0,0.0,0.0
4,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
# создадим объект класса NearestNeighbors
# metric = ‘cosine’: косинусное сходство; algorithm = ‘brute’: полный перебор; 
# n_neighbors = 20: количество соседей; n_jobs = -1: вычисления на всех свободных ядрах процессора
knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute', n_neighbors = 20, n_jobs = -1)
 
# обучим модель
knn.fit(csr_data)

,n_neighbors,20
,radius,1.0
,algorithm,'brute'
,leaf_size,30
,metric,'cosine'
,p,2
,metric_params,None
,n_jobs,-1


In [68]:
recommendations = 50
search_word = 'Code Geass'

In [69]:
anime_search = anime[anime['name'].str.contains(search_word)]
anime_search

,anime_id,name
13,2904,Code Geass: Hangyaku no Lelouch R2
19,1575,Code Geass: Hangyaku no Lelouch
574,6768,Code Geass: Hangyaku no Lelouch R2 Special Edi...
721,15197,Code Geass: Boukoku no Akito 2 - Hikisakareshi...
885,4596,Code Geass: Hangyaku no Lelouch Special Editio...
1132,8888,Code Geass: Boukoku no Akito 1 - Yokuryuu wa M...
1308,15199,Code Geass: Boukoku no Akito 3 - Kagayaku Mono...
1406,15201,Code Geass: Boukoku no Akito 4 - Nikushimi no ...
1441,23359,Code Geass: Boukoku no Akito 2 - Hikisakareshi...
2359,30711,Code Geass: Boukoku no Akito Final - Itoshiki ...


In [70]:
anime_id = anime_search.iloc[0]['anime_id']
 
# далее по индексу фильма в датасете найдем соответствующий индекс
# в матрице предпочтений
anime_id = user_item_matrix[user_item_matrix['anime_id'] == anime_id].index[0]
anime_id

np.int64(2157)

In [71]:
print(csr_data[anime_id])

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 14747 stored elements and shape (1, 24677)>
  Coords	Values
  (0, 2)	9.0
  (0, 6)	8.0
  (0, 7)	9.0
  (0, 10)	10.0
  (0, 11)	10.0
  (0, 13)	8.0
  (0, 14)	10.0
  (0, 15)	10.0
  (0, 16)	-1.0
  (0, 18)	8.0
  (0, 23)	9.0
  (0, 24)	9.0
  (0, 25)	10.0
  (0, 27)	10.0
  (0, 32)	10.0
  (0, 33)	7.0
  (0, 35)	9.0
  (0, 36)	8.0
  (0, 37)	9.0
  (0, 38)	7.0
  (0, 39)	9.0
  (0, 40)	8.0
  (0, 41)	10.0
  (0, 42)	9.0
  (0, 43)	7.0
  :	:
  (0, 24642)	9.0
  (0, 24643)	10.0
  (0, 24644)	10.0
  (0, 24645)	9.0
  (0, 24648)	9.0
  (0, 24649)	8.0
  (0, 24650)	10.0
  (0, 24651)	6.0
  (0, 24654)	7.0
  (0, 24655)	9.0
  (0, 24656)	10.0
  (0, 24657)	10.0
  (0, 24659)	8.0
  (0, 24660)	8.0
  (0, 24661)	-1.0
  (0, 24662)	9.0
  (0, 24663)	9.0
  (0, 24667)	10.0
  (0, 24668)	10.0
  (0, 24669)	9.0
  (0, 24670)	9.0
  (0, 24671)	10.0
  (0, 24672)	7.0
  (0, 24673)	10.0
  (0, 24676)	10.0


In [72]:
#поиск индексов ближайших соседей 
#массив индексов фильмов (indices) и массив расстояний (distances) до них
distances, indices = knn.kneighbors(csr_data[anime_id], n_neighbors = recommendations + 1)


In [73]:
# уберем лишние измерения через squeeze() и преобразуем массивы в списки с помощью tolist()
indices_list = indices.squeeze().tolist()
distances_list = distances.squeeze().tolist()
 
indices_distances = list(zip(indices_list, distances_list))
 
# в набор кортежей (tuple)
print(type(indices_distances[0]))
 
# и посмотрим на первые три пары/кортежа
print(indices_distances[:3])

<class 'tuple'>
[(2157, 0.0), (1349, 0.053814903889234156), (1316, 0.27916102409508226)]


In [74]:
indices_distances_sorted = sorted(indices_distances, key = lambda x: x[1], reverse = False)

# и убрать первый элемент с индексом 493 (потому что это и есть "Матрица")
indices_distances_sorted = indices_distances_sorted[1:]
indices_distances_sorted

[(1349, 0.053814903889234156),
 (1316, 0.27916102409508226),
 (2782, 0.3396325363914372),
 (3068, 0.35924027800538105),
 (1652, 0.3692234579698148),
 (3506, 0.37071005876627405),
 (199, 0.3711903841347677),
 (98, 0.38180047961134567),
 (1668, 0.383457799327508),
 (3976, 0.3859306408398979),
 (2605, 0.386386064512091),
 (4444, 0.38799445220497597),
 (1751, 0.392763648644331),
 (3324, 0.40460662389443247),
 (3121, 0.40519979390613914),
 (323, 0.4099214607676136),
 (740, 0.4171284451248223),
 (3660, 0.41825741183532483),
 (2591, 0.4218168147999708),
 (2776, 0.42367756768399467),
 (3792, 0.42585963710715924),
 (10, 0.43241167364016664),
 (2414, 0.4378634590147472),
 (3617, 0.43951278768677826),
 (3970, 0.4422295146554778),
 (18, 0.4439105106140263),
 (3838, 0.44731320424089094),
 (4697, 0.4495211760278779),
 (3910, 0.46222827174089975),
 (4182, 0.4640749566977945),
 (3145, 0.46431792212141243),
 (3076, 0.4648305836421023),
 (1042, 0.46710276317903254),
 (777, 0.4691429636363861),
 (3636, 0

In [75]:
recom_list = []

# теперь в цикле будем поочередно проходить по кортежам
for ind_dist in indices_distances_sorted:

    # искать movieId в матрице предпочтений
    matrix_anime_id = user_item_matrix.iloc[ind_dist[0]]['anime_id']

    # выяснять индекс этого фильма в датафрейме movies
    id = anime[anime['anime_id'] == matrix_anime_id].index

    # брать название фильма и расстояние до него
    name = anime.iloc[id]['name'].values[0]
    dist = ind_dist[1]

    # помещать каждую пару в питоновский словарь
    # который, в свою очередь, станет элементом списка recom_list
    recom_list.append({'Name' : name, 'Distance' : dist})

In [76]:
recom_list[:5]

[{'Name': 'Code Geass: Hangyaku no Lelouch', 'Distance': 0.053814903889234156},
 {'Name': 'Death Note', 'Distance': 0.27916102409508226},
 {'Name': 'Fullmetal Alchemist: Brotherhood', 'Distance': 0.3396325363914372},
 {'Name': 'Angel Beats!', 'Distance': 0.35924027800538105},
 {'Name': 'Tengen Toppa Gurren Lagann', 'Distance': 0.3692234579698148}]

In [77]:
#преобразование в датафрейм
recom_df = pd.DataFrame(recom_list, index = range(1, recommendations + 1))
recom_df

,Name,Distance
1,Code Geass: Hangyaku no Lelouch,0.053815
2,Death Note,0.279161
3,Fullmetal Alchemist: Brotherhood,0.339633
4,Angel Beats!,0.359240
5,Tengen Toppa Gurren Lagann,0.369223
6,Steins;Gate,0.370710
7,Elfen Lied,0.371190
8,Fullmetal Alchemist,0.381800
9,Darker than Black: Kuro no Keiyakusha,0.383458
10,Sword Art Online,0.385931
